In [ ]:
import sys
import os
from os.path import join
import numpy as np
import h5py
import matplotlib.pyplot as plt
import proplot as pplt

pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

In [ ]:
datadir = './Diagnostics/Data/Measurements/2022-05-09/'
filenames = os.listdir(datadir)
filenames

In [ ]:
filename = '220509101220-scan1d.h5'
cam = 'Cam06'
ny = 512  # number of y pixels
nx = 612  # number of x pixels

In [ ]:
file = h5py.File(join(datadir, filename), 'r')

In [ ]:
# Errors and warnings from log
for i in range(file['log'].size):
    if not(file['/log'][i, 'level'] == 'INFO'.encode('utf')):
        timestr = datetime.fromtimestamp(file['/log'][0, 'timestamp']).strftime("%m/%d/%Y, %H:%M:%S")
        print(f"{timestr} {file['log'][i, 'message']}")

# Configuration data
for key in file['/config'].keys():
    print(f"{key}")
    print("--------------")
    for name in file['/config'][key].dtype.names:
        print(f"{name}: {file['config'][key][name]}")
    print()

In [ ]:
data = file['/scandata']
attrs = data.dtype.names
acts = [key for key in attrs if 'PositionSync' in key]
print('attrs:', attrs)
print('acts:', acts)

Extract the vertical beam profile.

In [ ]:
idx = list(range(0, len(data), 5))
ncols = 6
nrows = int(np.ceil(len(idx) / ncols))

fig, axes = pplt.subplots(ncols=ncols, nrows=nrows, figwidth=10.0)
axes.format(xticks=[], yticks=[])
for i, ax in zip(idx, axes):
    im = data[i, cam + '_Image'].reshape(ny, nx)
    profx = data[i, cam + '_ProfileX']
    profy = data[i, cam + '_ProfileY']
    cut = 100
    ax.plot(profx[cut:-cut], color='black')
#     axes[i].imshow(im)
plt.show()

In [ ]:
fig, ax = pplt.subplots()
ax.plot(data['y_PositionSync'], data['Cam06_Integral'], marker='.', color='black')
ax.format(xlabel='y_PositionSync', ylabel='Cam06_Integral')

## 2022-05-09 experiment

In [ ]:
datadir = './Diagnostics/Data/Measurements/2022-05-09/'
filenames = os.listdir(datadir)
filenames = [f for f in filenames if 'scan1d' in f]
filenames

In [ ]:
def get_timestamp(filename):
    tstr = filename.split('-')[0]
    year, month, day, hour, minute, second = [int(tstr[i:i+2]) for i in range(0, len(tstr), 2)]
    return datetime(year, month, day, hour, minute, second)

filenames = sorted(filenames, key=get_timestamp)
filenames

In [ ]:
# The first four measurements are for the first set of optics as the beam current is increased.
groups = [[], []]
for i in range(2):
    lo = 4 * i
    hi = lo + 4
    for filename in filenames[lo:hi]:   
        file = h5py.File(join(datadir, filename), 'r')
        data = file['scandata']
        groups[i].append(data)

In [ ]:
fig, axes = pplt.subplots(nrows=4, ncols=1, figsize=(4.0, 5.0))
for j, (group, alpha) in enumerate(zip(reversed(groups), [1.0, 0.2])):
    for i, (ax, data) in enumerate(zip(axes, group)):
#         ax.set_title(f"bcm04 = {np.mean(data['bcm04']):.2f}")
        text_kws = dict()
        if j == 0:
            text_kws['xy'] = (0.02, 0.95)
            text_kws['alpha'] = 1.0
        else:
            text_kws['xy'] = (0.02, 0.80)
            text_kws['alpha'] = 0.3
        ax.annotate(f"bcm04 = {np.mean(data['bcm04']):.2f}", xycoords='axes fraction', 
#                         fontsize='small',
                    horizontalalignment='left', verticalalignment='top', **text_kws)
        x = data['y_PositionSync']
        y = data['Cam06_Integral']
        y = y / np.sum(y)
#         label = f"bcm04 = {np.mean(data['bcm04']):.2f}"
        label = ['Default optics', 'Modified optics'][j]
        ax.plot(x, y, color='black', alpha=alpha, marker='.', ms=3, label=label)
        
# Plot a vertical line representing slit position.
# y_slit_pos = 13.0
# for ax in axes:
#     ax.axvline(y_slit_pos, alpha=0.3)

axes[0].legend(ncols=2, loc='t', framealpha=0.0)
axes.format(xlabel='y_PositionSync [mm]', ylabel='Cam06_Integral (normalized)', yticklabels=[])
# axes.format(rightlabels=['', '', r'$\leftarrow$ Intensity', ''])
plt.savefig('_output/full_projections')
plt.show()

In [ ]:
x = np.arange(nx)
cut = 100
idx = x[cut:-cut]
fig, axes = pplt.subplots(nrows=2, ncols=len(group), figsize=(8.0, 3.0))
for row, group in enumerate(groups):  # new optics, then old optics
    for ax, data in zip(axes[row, :], group):
        m = ax.pcolormesh(
            x[idx], 
            data['y_PositionSync'],
            data['Cam06_ProfileX'][:, idx],
#             colorbar=True,
#             discrete=True,
            cmap='dusk_r',
        )
# fig.colorbar(m)
for ax, data in zip(axes[0, :], groups[0]):
    ax.set_title(f"bcm04 = {np.mean(data['bcm04']):.2f}")
axes.format(
    xlabel='x screen [pixel]', ylabel='y slit [mm]',
    leftlabels=['Modified optics', 'Default optics'],
    suptitle=r'Beam current $\rightarrow$',
)
plt.savefig('_output/y_vs_dE.png')